# Tick Strategy
_Version 0.1_

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# load our helper functios
import simtools as st
import tick as tk
from getstock import *
from simtools import *

matplotlib.rcParams[ 'figure.figsize' ] = ( 16, 9 )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load TAQ Data

We'll assume we have already done some cleaning by using our Tick Data Setup or similar code

In [6]:
# set up some variables
# NOTE: tickfile is a merged TAQ file!!!
tickfile = 'APPL-TAQ-Sep03.zip'
#test_day = '09/03/19'
taq = pd.read_pickle(tickfile)

In [12]:
taq.iloc[200000:300000,:]

# filter on date
#day = taq[test_day]

,exch,bid_px,bid_size,ask_px,ask_size,qu_cond,qu_seqnum,natbbo_ind,qu_cancel,qu_source,symbol,suffix_x,suffix_y,trade_size,trade_px
2019-09-03 09:51:06.172355425,K,206.56,1.0,206.68,2.0,R,2612405.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 09:51:06.172592265,C,206.57,1.0,217.14,1.0,R,2612408.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 09:51:06.175154421,Q,206.56,2.0,206.60,3.0,R,2612434.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 09:51:06.175242516,N,206.57,4.0,206.61,4.0,R,2612435.0,2.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 09:51:06.175310952,N,206.57,1.0,206.61,4.0,R,2612436.0,2.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-03 10:03:41.046196783,N,205.68,1.0,205.73,4.0,R,4037948.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 10:03:41.046295422,N,205.67,4.0,205.73,4.0,R,4037949.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 10:03:41.046340089,C,205.69,1.0,217.14,1.0,R,4037950.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN
2019-09-03 10:03:41.046347489,P,205.67,1.0,205.73,1.0,R,4037951.0,0.0,NaN,N,AAPL,NaN,NaN,NaN,NaN


## Run the Simulator

In [194]:
# without the tick signal...
results = tk.algo_loop( taq.iloc[200000:230000,:] )

2019-10-28 17:43:59:490290: Beginning Tick Strategy run
Average stock spread for sample: 2.3477
2019-10-28 17:43:59:519977: starting main loop
2019-10-28 17:44:14:269323: end simulation loop
2019-10-28 17:44:14:269480: order analytics
2019-10-28 17:44:14:278335: Algo run complete.


In [195]:
results.keys()

dict_keys(['trades', 'total_pnl'])

In [196]:
trades = results['trades']

In [197]:
results['total_pnl']

7147876.10886674

In [198]:
trades.iloc[10]

current_tick                -1
stock_price              206.6
fair_value             206.596
current_position             9
current_pnl              -0.18
avg_price              206.618
actual_trade_price      206.61
actual_trade_shares         -1
trade_type                   a
trade_side                   s
Name: 2019-09-03 09:51:06.760476961, dtype: object

In [199]:
trades.head(60)

,current_tick,stock_price,fair_value,current_position,current_pnl,avg_price,actual_trade_price,actual_trade_shares,trade_type,trade_side
2019-09-03 09:51:06.743239357,1,206.61,207.705,1,0,0,206.61,1,a,b
2019-09-03 09:51:06.743297551,1,206.61,207.712,2,0,206.61,206.61,1,a,b
2019-09-03 09:51:06.743324465,1,206.61,207.718,3,0,206.61,206.61,1,a,b
2019-09-03 09:51:06.743598060,1,206.62,207.724,4,0.03,206.61,206.61,1,a,b
2019-09-03 09:51:06.743601754,1,206.62,207.729,5,0.04,206.61,206.61,1,a,b
2019-09-03 09:51:06.743623559,1,206.62,207.734,6,0.05,206.61,206.61,1,a,b
2019-09-03 09:51:06.743646196,1,206.62,207.753,7,0.06,206.61,206.63,1,a,b
2019-09-03 09:51:06.743650132,1,206.62,207.757,8,0.05,206.613,206.63,1,a,b
2019-09-03 09:51:06.743683851,1,206.62,207.76,9,0.04,206.615,206.63,1,a,b
2019-09-03 09:51:06.747934052,1,206.62,207.764,10,0.03,206.617,206.63,1,a,b


In [187]:
for key in list(trade_stats.keys())[1:]:
    print(key, trade_stats[key], end='\n')

max_long_Position 163.0
max_short_Position -41.0
ending_Position -24.0
best_PNL 56.56464974909824
worst_PNL -60.039349380359454
total_PNL -2745.8752936331693
